# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

import re
import numpy as np
import pandas as pd
import sqlite3
from sqlalchemy import create_engine
import pickle

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])


import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql('DisasterResponse', engine)

# Message column
X = df['message'] 
display(X)
# Classification label
Y = df.iloc[:, 4:]
display(Y)

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
7,1,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """
    This function performs the following steps:
    1. Detects and replaces URLs in the text with the placeholder "urlplaceholder".
    2. Tokenizes the text into individual words (tokens).
    3. Lemmatizes each token to its base form, converts it to lowercase, and strips any surrounding whitespace.
    
    Args:
        text (str): The input text to be processed.

    Returns:
        list: A list of clean, lemmatized tokens ready for further processing.
    """
    
    # Detecting URLs
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    # Tokenize and lemmatize
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline ([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
# Spliting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42)

# Fiting the Random Forest Classifier 
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
Y_pred = pipeline.predict(X_test)
    
# Report for the first model
print("=== Classification Report for the First Model (Y_pred) ===")
report_1 = classification_report(Y_test, Y_pred, target_names=Y_test.columns)
print(report_1)

=== Classification Report for the First Model (Y_pred) ===
                        precision    recall  f1-score   support

               related       0.81      0.93      0.87      4944
               request       0.82      0.34      0.48      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.73      0.53      0.61      2670
          medical_help       0.55      0.07      0.12       535
      medical_products       0.71      0.08      0.14       344
     search_and_rescue       0.67      0.09      0.16       159
              security       0.25      0.01      0.02       116
              military       0.49      0.10      0.16       200
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.16      0.27       418
                  food       0.90      0.38      0.53       745
               shelter       0.84      0.24      0.37       581
              clothing       0.86      0.06 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [7]:
parameters = {
    # CountVectorizer Parameters
    'vect__ngram_range': [(1, 2)],
    
    # TF-IDF Parameters
    #'tfidf__use_idf': [True, False],
    
    # Classifier Parameters
    'clf__estimator__min_samples_split': [3],
    'clf__estimator__n_estimators': [100, 150]

}

# Initializing GridSearchCV
cv = GridSearchCV(
    estimator=pipeline, 
    param_grid=parameters, 
    cv=2,  # 2-fold cross-validation
    verbose=3, 
    n_jobs=-1  # Use all available cores
)

# Fitting the Grid Search
cv.fit(X_train, Y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, vect__ngram_range=(1, 2) 
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, vect__ngram_range=(1, 2) 
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=150, vect__ngram_range=(1, 2) 
[CV] clf__estimator__min_samples_split=3, clf__estimator__n_estimators=150, vect__ngram_range=(1, 2) 
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, vect__ngram_range=(1, 2), score=0.24188790560471976, total=21.0min
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=100, vect__ngram_range=(1, 2), score=0.2385311768894314, total=21.4min
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=150, vect__ngram_range=(1, 2), score=0.2419896246566982, total=26.3min
[CV]  clf__estimator__min_samples_split=3, clf__estimator__n_estimators=150, vect__ngram_range=(1, 2), 

[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 27.7min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'vect__ngram_range': [(1, 2)], 'clf__estimator__min_samples_split': [3], 'clf__estimator__n_estimators': [100, 150]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [8]:
best_params = cv.best_params_
best_score = cv.best_score_

print("Best Parameters:", best_params)
print("Best F1 Score:", best_score)

Best Parameters: {'clf__estimator__min_samples_split': 3, 'clf__estimator__n_estimators': 150, 'vect__ngram_range': (1, 2)}
Best F1 Score: 0.240311260299


In [9]:
# Predict on the test set using the best estimator
model_best_params = cv.best_estimator_

Y_pred_best = model_best_params.predict(X_test)

In [10]:
def compare_results(Y_test, Y_pred, Y_pred_best, category_names):
    """
    Compare the classification reports of two models' predictions.
    
    Args:
        Y_test (DataFrame): True labels for the test set.
        Y_pred (numpy array): Predictions from the first model.
        Y_pred_best (numpy array): Predictions from the second model (e.g., best-tuned model).
        category_names (list): List of category names for the multi-output classification.
    
    Returns:
        None: Prints classification reports for both models.
    """
    # Report for the first model
    print("=== Classification Report for the First Model (Y_pred) ===")
    report_1 = classification_report(Y_test, Y_pred, target_names=category_names)
    print(report_1)
    
    # Report for the second (best) model
    print("\n=== Classification Report for the Best Model (Y_pred_best) ===")
    report_2 = classification_report(Y_test, Y_pred_best, target_names=category_names)
    print(report_2)

# Example usage:
# Assuming Y_test, Y_pred, and Y_pred_best are defined and category names are obtained from Y_test.columns
compare_results(Y_test, Y_pred, Y_pred_best, category_names=Y_test.columns.tolist())

=== Classification Report for the First Model (Y_pred) ===
                        precision    recall  f1-score   support

               related       0.81      0.93      0.87      4944
               request       0.82      0.34      0.48      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.73      0.53      0.61      2670
          medical_help       0.55      0.07      0.12       535
      medical_products       0.71      0.08      0.14       344
     search_and_rescue       0.67      0.09      0.16       159
              security       0.25      0.01      0.02       116
              military       0.49      0.10      0.16       200
           child_alone       0.00      0.00      0.00         0
                 water       0.87      0.16      0.27       418
                  food       0.90      0.38      0.53       745
               shelter       0.84      0.24      0.37       581
              clothing       0.86      0.06 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF
* compare with other model

In [11]:
class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """
    A custom transformer to check if a text starts with a verb.

    This can be useful in text classification problems, as messages that start with 
    verbs might have different meanings (e.g., action-oriented messages in disaster response).
    """  
    
    def starting_verb(self, text):
        """
        Checks if the first word in a sentence is a verb.
        
        Args:
        text (str): The input text.

        Returns:
        bool: True if the first word is a verb (VB, VBP) or 'RT' (retweet), otherwise False.
        """
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)
    

def model_pipeline():
    """
    Builds a machine learning pipeline with multiple feature extraction methods and a classifier.
    
    Returns:
        pipeline: A scikit-learn Pipeline object.
    """
    pipeline = Pipeline([
        ('features', FeatureUnion([            
            # Text preprocessing pipeline
            ('text_pipeline', Pipeline([
                ('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer())
            ])),
            # Custom feature extractor
            ('starting_verb', StartingVerbExtractor())
        ])),
        # Classifier
        ('clf', MultiOutputClassifier(RandomForestClassifier(min_samples_split=3, n_estimators= 150)))
    ])
    
    return pipeline


In [12]:
extra_model = model_pipeline()
extra_model.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

In [13]:
Y_pred_extra = extra_model.predict(X_test)

compare_results(Y_test, Y_pred_best, Y_pred_extra, category_names=Y_test.columns.tolist())

=== Classification Report for the First Model (Y_pred) ===
                        precision    recall  f1-score   support

               related       0.80      0.97      0.87      4944
               request       0.87      0.46      0.60      1111
                 offer       0.00      0.00      0.00        33
           aid_related       0.79      0.60      0.68      2670
          medical_help       0.70      0.06      0.11       535
      medical_products       0.70      0.08      0.15       344
     search_and_rescue       0.71      0.06      0.12       159
              security       0.50      0.01      0.02       116
              military       0.70      0.04      0.07       200
           child_alone       0.00      0.00      0.00         0
                 water       0.90      0.33      0.49       418
                  food       0.88      0.53      0.66       745
               shelter       0.85      0.32      0.46       581
              clothing       0.80      0.04 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 9. Export your model as a pickle file

In [14]:
pickle.dump(model_best_params, open('best_model.pkl', 'wb'))

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.